# Decoding with Belief Propagation

We want to iterate the following the following equations:

$$ \hat{m}_{\mu j} = \tanh ( \beta(\rho) J_\mu ) \prod_{ l \in {\cal L} (\mu) \backslash j } m_{\mu l}  \; \; ,  $$ 

$$ m_{\mu j} = \tanh \left(  \sum_{ \nu \in {\cal M} (j) \backslash \mu } \ \tanh^{-1} \left(  \hat{m}_{\nu j}  \right)    + \beta(\rho_\xi)  \right)   \; \; ,  $$  

which are Eqs.(96) from the paper [Low-density parity-check codes—A statistical physics perspective](https://www.sciencedirect.com/science/article/pii/S1076567002800180), by R. Vicente, D. Saad and Y. Kabashima.

The function $ \beta(x) $ is the Nishimori temperature,

$$ \beta(x) = \frac{1}{2} \log \left(  \frac{1- \rho}{\rho}  \right) \; \; .  $$

The quantity $\rho$ is the flip probablility of the noisy channel (BSC),

$$ P ( J | J^{(0)} ) = (1 - \rho) \delta_{J, J^{(0)} } + \rho \delta_{J, -J^{(0)} }   \; \; , $$

whereas the prior distribution for each meassage bit is assumed to be

$$ P ( S_j ) = (1 - \rho_\xi) \delta_{+1, S_j }  +  \rho_\xi \delta_{-1, S_j }  \; \; .  $$

The object $ {\cal L} (\mu)$ represents the set of $K$ non-zero elements on the row $\mu$ of the code generator matrix ${\cal G}$ (the one which adds redundancy), 

$$  {\cal L} (\mu) = \langle i_1, i_2, ..., i_K \rangle  \; \; .  $$

The are $C$ non-zero elements per column on the matrix ${\cal G}$:

$$ \sum_{\mu : j \in {\cal L}(\mu)} i_j = C \; \; ; \; \; \forall j = 1, ..., K \; \; . $$

The object $ {\cal M} (j)$ represents the set of all index sets that contain $j$.

After convergence of the iterative procedure, we can calculate the pseudo-posterior:

$$ m_{j} = \tanh \left(  \sum_{ \nu \in {\cal M} (j) } \tanh^{-1} \left(  \hat{m}_{\nu j}  \right)    + \beta(\rho_\xi)  \right)   \; \; ,  $$
from which the Bayes optimal estimate is obtained
$$ \hat{\xi}_j  = sign( m_j ) $$ 

In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt

Defining the variables of the problem:

In [26]:
# Message lengh
N = 100

# Codeword lengh
M = 200

# Non-zero elements per row of the generation matrix
K = 4

# Number of messages
n = 10

# Noisy channel
p = 0.5
beta = 0.5*np.log( (1 - p) / p)

# Message prior
p_prior = 0.1
beta_prior = 0.5*np.log( (1 - p_prior) / p_prior)

## Generating messages

Each message is a $N$ dimensional vector. Generate a set of $n$ messages.

In [27]:
random = torch.rand([n, N])
    
message = torch.zeros([n, N])

for j in range(random.shape[0]):
    
    for k in range(random.shape[1]):
               
            #### -1 with probability p_prior
            if random[j,k] <= p_prior:
                message[j,k] = -1.
            #### +1 with probability 1 - p_prior
            else:
                message[j,k] = +1.

In [28]:
message.shape

torch.Size([10, 100])

## Encoding

Each message is encoded to a high dimensional vector ${\bf J}^{(0)} \in \{ \pm 1  \}^M$ defined as 

$$   J^{(0)}_{\langle i_1, i_2, ...., i_K \rangle} = \xi_1 \xi_ 2 ... \xi_K  \; \; ,$$

where $M$ sets of $K \in [ 1, ..., N]$ indexes are randomly chosen.

In [29]:
encoding = torch.randint(0, N, [M, K])

In [30]:
encoding.shape

torch.Size([200, 4])

From `encoding`, we construct the encoded message ${\bf J}^{(0)}$.

In [31]:
#for j in range(message.shape[0]):
    
#    J0 = torch.take(message[j], encoding).prod(dim=1)

In [32]:
# Initializing
J0 = torch.take(message[0], encoding).prod(dim=1)
J0 = J0.unsqueeze(0)

for j in range(1, message.shape[0]):
    
    J0_ = torch.take(message[j], encoding).prod(dim=1)
    J0_ = J0_.unsqueeze(0)
    
    J0 = torch.cat((J0, J0_), dim= 0)

## Corrupted version of the encoded set of messages

In [33]:
J = J0.clone()

In [34]:
random = torch.rand(J.shape)
                      
for j in range(J.shape[0]):
    for k in range(J.shape[1]):
          
        if random[j, k] <= p:
            J[j, k] = -J[j, k]

In [50]:
J

tensor([[-1., -1.,  1.,  ..., -1.,  1., -1.],
        [ 1.,  1., -1.,  ..., -1., -1.,  1.],
        [-1.,  1., -1.,  ..., -1., -1., -1.],
        ...,
        [ 1.,  1.,  1.,  ...,  1., -1.,  1.],
        [ 1.,  1., -1.,  ..., -1., -1.,  1.],
        [ 1., -1., -1.,  ...,  1., -1., -1.]])

In [52]:
J.shape

torch.Size([10, 200])

## Iterating BP equations for one message

Let us focus in one received message to iterate the belief propagation equations.

$$ \hat{m}_{\mu j} = \tanh ( \beta(\rho) J_\mu ) \prod_{ l \in {\cal L} (\mu) \backslash j } m_{\mu l}  \; \; ,  $$ 

$$ m_{\mu j} = \tanh \left(  \sum_{ \nu \in {\cal M} (j) \backslash \mu} \ \tanh^{-1} \left(  \hat{m}_{\nu j}  \right) + \beta(\rho_\xi)    \right)   \; \; ,  $$  

with $j = 1, ..., N$ and $\mu = 1, ..., M$.

We cal this message `J_`. We will worry later about a loop over all the received messages.

In [35]:
J_ = J[0]
print(J_)
print(J_.shape)

tensor([-1., -1.,  1.,  1.,  1., -1., -1., -1., -1.,  1.,  1.,  1., -1.,  1.,
         1., -1.,  1., -1., -1., -1., -1.,  1.,  1., -1., -1.,  1., -1.,  1.,
        -1., -1., -1.,  1.,  1., -1., -1., -1.,  1.,  1., -1., -1., -1., -1.,
        -1.,  1., -1., -1.,  1.,  1., -1., -1.,  1., -1.,  1.,  1., -1.,  1.,
        -1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1., -1., -1.,  1., -1., -1.,
        -1.,  1., -1., -1.,  1., -1., -1.,  1., -1.,  1.,  1.,  1., -1., -1.,
         1.,  1., -1., -1.,  1.,  1., -1.,  1., -1.,  1.,  1.,  1., -1., -1.,
        -1., -1.,  1.,  1., -1., -1.,  1.,  1., -1., -1., -1.,  1.,  1., -1.,
         1.,  1., -1., -1., -1., -1., -1.,  1.,  1., -1., -1.,  1.,  1.,  1.,
         1.,  1., -1.,  1.,  1., -1., -1., -1., -1., -1., -1.,  1., -1., -1.,
         1.,  1.,  1., -1., -1., -1.,  1.,  1., -1., -1.,  1., -1., -1.,  1.,
         1.,  1.,  1.,  1., -1.,  1., -1., -1.,  1.,  1.,  1.,  1.,  1., -1.,
         1., -1., -1.,  1., -1.,  1.,  1.,  1., -1., -1.,  1., -

Random initialization of the beliefs $m_{\mu l}$.

In [36]:
m = torch.rand(M, N)

In [37]:
m

tensor([[0.2830, 0.1804, 0.0283,  ..., 0.8417, 0.7593, 0.6501],
        [0.4263, 0.6466, 0.4386,  ..., 0.8231, 0.2743, 0.2494],
        [0.7610, 0.6929, 0.3876,  ..., 0.3168, 0.2898, 0.5129],
        ...,
        [0.1362, 0.8182, 0.2399,  ..., 0.7423, 0.7903, 0.2556],
        [0.7293, 0.5451, 0.6052,  ..., 0.1215, 0.8149, 0.5406],
        [0.1735, 0.7988, 0.3331,  ..., 0.9662, 0.1540, 0.7476]])

Initialize an empty tensor to represent $\hat{m}_{\mu l}$.

In [38]:
m_hat = torch.empty(M, N)

In [39]:
m_hat

tensor([[2.0823e+23, 4.2186e-08, 2.6807e-09,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        ...,
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00]])

We want to calculate $\hat{m}_{\mu j}$.

$$ \hat{m}_{\mu j} = \tanh ( \beta(\rho) J_\mu ) \prod_{ l \in {\cal L} (\mu) \backslash j } m_{\mu l}  \; \; ,  $$ 

This first implementation has two `for` loops. This is potentially harmful if one cares about efficienty. We obviously do, but since we are just beginning, lets go on like this.

In [40]:
for mu in range(M):
    for j in range(N):
          
        # Keep only L(mu) which a are different of j
        index_no_j = torch.nonzero(encoding[mu] != j).squeeze()
        L_no_j = encoding[mu][index_no_j]
        
        # Message update        
        m_hat[mu, j] = torch.tanh( beta* J_[mu])*torch.take(m[mu], L_no_j).prod(dim=0)

In [41]:
m_hat

tensor([[-0., -0., -0.,  ..., -0., -0., -0.],
        [-0., -0., -0.,  ..., -0., -0., -0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [-0., -0., -0.,  ..., -0., -0., -0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [-0., -0., -0.,  ..., -0., -0., -0.]])

The next step is to implement:

$$ m_{\mu j} = \tanh \left(  \sum_{ \nu \in {\cal M} (j) \backslash \mu} \ \tanh^{-1} \left(  \hat{m}_{\nu j}  \right) + \beta(\rho_\xi)     \right)  \; \; ,  $$ 

In [42]:
for j in range(N):
    
    for mu in range(M):
        
        M_set = torch.where(encoding == j)[0]
        
        M_set_no_mu = M_set[torch.nonzero(M_set != mu).squeeze()]
        
        m[mu, j] = torch.tanh(torch.take(np.arctanh(m_hat[:, j]), M_set_no_mu).sum() + beta_prior)

In [43]:
m

tensor([[0.8000, 0.8000, 0.8000,  ..., 0.8000, 0.8000, 0.8000],
        [0.8000, 0.8000, 0.8000,  ..., 0.8000, 0.8000, 0.8000],
        [0.8000, 0.8000, 0.8000,  ..., 0.8000, 0.8000, 0.8000],
        ...,
        [0.8000, 0.8000, 0.8000,  ..., 0.8000, 0.8000, 0.8000],
        [0.8000, 0.8000, 0.8000,  ..., 0.8000, 0.8000, 0.8000],
        [0.8000, 0.8000, 0.8000,  ..., 0.8000, 0.8000, 0.8000]])

The next step is to implement the pseudo-posterior, which is calculated after convergence of the messages above:

$$ m_{j} = \tanh \left(  \sum_{ \nu \in {\cal M} (j) } \tanh^{-1} \left(  \hat{m}_{\nu j}  \right)    + \beta(\rho_\xi)  \right)   \; \; ,  $$

In [44]:
m_bayes = []

for j in range(N):
    
    M_set = torch.where(encoding == j)[0]
       
    m_j = torch.tanh(torch.take(np.arctanh(m_hat[:, j]), M_set).sum() + beta_prior)
    
    m_bayes.append(m_j.item())
            
m_bayes

[0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,


In [45]:
len(m_bayes)

100

In [46]:
np.sign(m_bayes)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

Gathering all message iterations on functions:

In [167]:
def m_to_mhat(N, M, J_, m, beta, encoding):
    
    m_hat = torch.empty(M, N)
    
    for mu in range(M):
        for j in range(N):
                     
            # Keep only L(mu) which a are different of j
            index_no_j = torch.nonzero(encoding[mu] != j).squeeze()
            L_no_j = encoding[mu][index_no_j]
        
            # Message update        
            m_hat[mu, j] = torch.tanh( beta* J_[mu])*torch.take(m[mu], L_no_j).prod(dim=0)
            
    return m_hat


#####################################################################


def mhat_to_m(N, M, m_hat, beta_prior, encoding):
    
    m = torch.empty(M, N)
    
    for j in range(N):
        
        for mu in range(M):
            
            M_set = torch.where(encoding == j)[0]
        
            M_set_no_mu = M_set[torch.nonzero(M_set != mu).squeeze()]
            
            m[mu, j] = torch.tanh(torch.take(np.arctanh(m_hat[:, j]), M_set_no_mu).sum() + beta_prior)
            
    return m


#####################################################################


def m_bayes(N, m_hat_final, beta_prior, encoding):
    
    m_bayes = []

    for j in range(N):
           
        M_set = torch.where(encoding == j)[0]
  
        m_j = torch.tanh(torch.take(np.arctanh(m_hat_final[:, j]), M_set).sum() + beta_prior)
    
        m_bayes.append(m_j.item())
        
    return m_bayes

Now we construct a function for the iterative decoding.

In [168]:
def BP_LDPC(N, M, J_, beta, beta_prior, encoding, message, num_it= 100, verbose= 1):
     
    m_hat = torch.rand(M, N)
    m = torch.empty(M, N)
    
    for j in range(num_it):
        
        print('--it = %d' % j)
              
        m = mhat_to_m(N, M, m_hat, beta_prior, encoding)
        m_hat = m_to_mhat(N, M, J_, m, beta, encoding)
        
        ## Monitoring performace
        if (j % verbose) == 0:
            m_ = m_bayes(N, m_hat, beta_prior, encoding)
            print('overlap= ', torch.dot(message, torch.Tensor(np.sign(m_))).item() / N)
        
    m_final = m_bayes(N, m_hat, beta_prior, encoding)
    
    return np.sign(m_final)

In [49]:
opt_dec_Bayes = BP_LDPC(N, M, J_, beta, beta_prior, encoding, message[0], num_it= 20, verbose= 1)

--it = 0
overlap=  0.76
--it = 1
overlap=  0.76
--it = 2
overlap=  0.76
--it = 3
overlap=  0.76
--it = 4
overlap=  0.76
--it = 5
overlap=  0.76
--it = 6
overlap=  0.76
--it = 7
overlap=  0.76
--it = 8
overlap=  0.76
--it = 9
overlap=  0.76
--it = 10
overlap=  0.76
--it = 11
overlap=  0.76
--it = 12
overlap=  0.76
--it = 13
overlap=  0.76
--it = 14
overlap=  0.76
--it = 15
overlap=  0.76
--it = 16
overlap=  0.76
--it = 17
overlap=  0.76
--it = 18
overlap=  0.76
--it = 19
overlap=  0.76


Ok. It is still slow (we will worrie about this latter), but it appears to work.

Observe that we arbitrarily consider a certain `num_it`. Naturally, a important question remains: how do we monitor the convergence of BP?

## Quick test on previous codes and messages

In [157]:
# Message lengh
N = 250

# Codeword lengh
M = 500

# Non-zero elements per row of the generation matrix
K = 4

# Number of messages
n = 100

# Noisy channel
p = 0.3
beta = 0.5*np.log( (1 - p) / p)

# Message prior
p_prior = 0.01
beta_prior = 0.5*np.log( (1 - p_prior) / p_prior)

In [136]:
encoding = torch.load('codes/code_N_250_M_500_K_4_p_03_p_prior_001_j_0.pt')
message = torch.load('codes/message_N_250_M_500_K_4_p_03_p_prior_001.pt')

In [137]:
encoding

tensor([[ 1.,  1.,  1.,  ...,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  ...,  1., -1.,  1.],
        [ 1.,  1.,  1.,  ...,  1.,  1.,  1.],
        ...,
        [ 1.,  1.,  1.,  ...,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  ...,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  ...,  1.,  1.,  1.]])

In [145]:
print(message)
message.shape

tensor([[ 1.,  1.,  1.,  ...,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  ...,  1., -1.,  1.],
        [ 1.,  1.,  1.,  ...,  1.,  1.,  1.],
        ...,
        [ 1.,  1.,  1.,  ...,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  ...,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  ...,  1.,  1.,  1.]])


torch.Size([100, 250])

Well, we made a mistake saving the codes. We have actually saved the messages. That is a problem. For now, let us generate a new code.

In [139]:
encoding  = torch.randint(0, N, [M, K])
encoding

tensor([[ 94,  21, 204, 242],
        [162,  49, 100, 151],
        [230, 198, 123,  57],
        ...,
        [ 87, 176,   1, 211],
        [ 99,  72,  51,  91],
        [248, 145, 105, 154]])

Encoding.

In [140]:
# Initializing
J0 = torch.take(message[0], encoding).prod(dim=1)
J0 = J0.unsqueeze(0)

for j in range(1, message.shape[0]):
    
    J0_ = torch.take(message[j], encoding).prod(dim=1)
    J0_ = J0_.unsqueeze(0)
    
    J0 = torch.cat((J0, J0_), dim= 0)

In [144]:
print(J0)
J0.shape

tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        ...,
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]])


torch.Size([100, 500])

Corrupted version.

In [142]:
J = J0.clone()

random = torch.rand(J.shape)
                      
for j in range(J.shape[0]):
    for k in range(J.shape[1]):
          
        if random[j, k] <= p:
            J[j, k] = -J[j, k]

In [143]:
print(J)
J.shape

tensor([[-1., -1.,  1.,  ...,  1., -1.,  1.],
        [-1.,  1., -1.,  ...,  1.,  1.,  1.],
        [ 1., -1., -1.,  ..., -1.,  1.,  1.],
        ...,
        [-1.,  1.,  1.,  ..., -1.,  1.,  1.],
        [ 1.,  1., -1.,  ...,  1.,  1., -1.],
        [ 1.,  1.,  1.,  ...,  1., -1.,  1.]])


torch.Size([100, 500])

In [149]:
J[0].shape

torch.Size([500])

In [151]:
message[0].shape

torch.Size([250])

In [155]:
print(N)
print(M)
print(encoding.shape)
print(J.shape)
print(message.shape)

250
500
torch.Size([500, 4])
torch.Size([100, 500])
torch.Size([100, 250])


In [ ]:
overlap_list = []

for k in range(int(J.shape[0] / 10 )):
    
    print('----message %d' % k)
       
    opt_dec_Bayes = BP_LDPC(N, M, J[k], beta, beta_prior, encoding, message[k], num_it= 10, verbose= 1)
    
    overlap_list.append(opt_dec_Bayes)